</div>
<div align="left">
  <img src="img/abstract.png" width="400" alt="Funny little diagram">
  <p><em> Evolve nodes, evolve plans, and learn from the best performing ones.</em></p>
</div>
<div align="center">
</em></p>
</div>

#### Node Initialization (Refactoring ...)

In [1]:
from methods.llm import get_async_vllm_endpoint
import os 

endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]

get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) # Unlimited LLM endpoints

Could not load vllm class, check CUDA support and GPU RAM size


In [2]:
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.evolnode import EvolNode
from methods.llm import get_groq_response, get_claude_response

# Code + Compilor Task
# mp = MetaPrompt("Search for age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.CODE)
# Prompt + LLM Task
mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

test_inputs = [c[0] for c in test_cases]

node = EvolNode(mp, None, None, get_response=get_endpoint_response, test_cases=test_cases) # setting manual test cases

node.evolve("i1", replace=True, batch_size=20, num_runs=2, print_summary=True) # Scale up batch size

Processing queries: 100%|██████████| 26/26 [00:43<00:00,  1.68s/it]
Processing queries: 0it [00:00, ?it/s]

🏆 Best Code Performance Summary 🏆
  ⚡ Structural fitness: 0.33
  🎯 Functional fitness: 1.00
  ⭐ Global fitness:     0.67
  🔄 Batch size:        18
  ⏱️ Time taken: 115.86 seconds


📊 Code 0: Fitness: 0.0%
--------------------------------------------------------------------------------
❌ Error Messages:
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)
Failed to parse LLM response -- JsonDecodeError : 
Expecting value: line 1 column 1 (char 0)AstLiteralError : 
invalid syntax (<unknown>, line 1)


📊 Code 1: Fitness: 41.7%
--------------------------------------------------------------------------------
❌ Error Messages:
Input: {'name': 'Dilireba'}, prediction is not aligned with expected output, Expected: {'age': 32} Predicted: {'title': 'Dilraba Dilmurat - Wikipedia', 'link': 'https://en.wikipedia.org/wiki/Dilraba_Dilmurat', 'snippet': 'Dilraba Dilmurat (Uyghur: دىلرەبا دىلمۇرات, Chinese: 迪丽热

</div>
<div align="center">
  <img src="img/Project-Nirvana-evolve.gif" width="500" alt="Fourier reconstruction convergence">
  <p><em> Evolve a population of nodes. </em></p>
</div>

In [5]:
# Population building phase ... 
from methods.llm import get_groq_response, get_claude_response
from methods.meta_prompt import MetaPrompt, PromptMode
from methods.population import Evolution

mp = MetaPrompt("Get the age of a celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"], PromptMode.PROMPT) # 

test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

evo = Evolution(pop_size=3, meta_prompt=mp, get_response=get_endpoint_response, 
                test_cases=test_cases, max_attempts=3, num_eval_runs=2,
                load=True)

strategies = ["m2"] # ["i1", "i1", "m2", "e2"]
evo.get_offspring(strategies)

evo.chat("How effective is the current evolution strategy? What improvement has it made in terms of fitness, and in terms of the implementation?",
         get_claude_response) 

# code-based check 
print(evo.population_info)

Best code has structural fitness 0.33, functional fitness 0.50, global fitness 0.42
Based on the provided information, I'll analyze the effectiveness of the current evolution strategy:

1. Fitness Improvement:
- The current best fitness is 1.0 (from the initial population)
- The offspring solution has a fitness of approximately 0.417
- This indicates that the evolution strategy has not improved fitness, as the offspring's fitness is lower than the parent solutions

2. Implementation Improvements:

The offspring solution shows several implementation improvements compared to the parents:

Positive Changes:
1. Added actual Google search functionality through Serper API
2. Introduced error handling with try/except blocks
3. Added type hints for better code readability and maintainability
4. Included more structured API interaction with proper HTTP connection handling
5. Implemented JSON parsing and formatting of search results

Limitations/Issues:
1. The solution is more complex and potent


</div>
<div align="center">
  <img src="https://github.com/user-attachments/assets/af98faeb-66d6-4278-af86-67d668d1954e" width="900" alt="Fourier reconstruction convergence">
  <p><em> Plan, and evolve the plans. </em></p>
</div>


In [9]:
from methods.llm import get_async_vllm_endpoint
import os 

endpoint_id = "vllm-vbzs7iyoydoidq"
api_key = os.environ["RUNPOD_API_KEY"]

# get_endpoint_response = get_async_vllm_endpoint(endpoint_id, api_key) # Unlimited LLM endpoints

In [10]:
get_endpoint_response = get_async_vllm_endpoint_(endpoint_id, api_key)


In [11]:
from methods.llm import get_claude_response, get_groq_response
from methods.diagram import visualize_plan_dict
from methods.meta_prompt import MetaPlan
from methods.evolnode import PlanNode


# Initialize PlanNode 
mp = MetaPlan("Get the age of celebrity.", "get_celeb_age", ["name"], ["age"], ["str"], ["int"])
plan = PlanNode(mp, get_endpoint_response)

# i1 evolution of plan
plan_dicts, err_msg = plan.evolve_plan_dict(method="i1", batch_size=20) # Batch_size of 100 gives no slow-down

for plan_dict in plan_dicts:
    visualize_plan_dict(plan_dict, plan.meta_prompt.task)
    break

# Manual input on main-node test cases 
main_test_cases = [
    ({"name": "Dilireba"}, {"age": 32}),
    ({"name": "ChengXiao"}, {"age": 26})
]

is_sucess, err_msg = plan.spawn_test_cases(main_test_cases) #  pinned test cases generation

 :: Evolving 20 plans in parallel...


Processing queries: 100%|██████████| 20/20 [00:14<00:00,  1.36it/s]


Total time elapsed: 14.66s
 :: Pseudo-code generated for each plan


Processing queries: 100%|██████████| 20/20 [00:27<00:00,  1.38s/it]


Total time elapsed: 27.65s
 :: Plan_dict generated for each plan


success: successfully compiled d2_output/plan_graph.d2 to d2_output/plan_graph.png in 196.279625ms
Processing queries: 100%|██████████| 20/20 [00:24<00:00,  1.22s/it]

Total time elapsed: 24.44s
Spawned 2 test cases for all sub-nodes


In [5]:
plan.evolve_sub_nodes() # Doesn't make sense to take that long, if I am evoling one node at a time ???


# Bug above
# ---> 86 is_aligned, error_msg_delta = metric(pred_value, target_value)
#      87 if not is_aligned:
#      88     error_msg += error_msg_delta + "\n"

# TypeError: 'tuple' object is not callable

KeyboardInterrupt: 